In [1]:
import numpy as np
from astropy.table import Table
import pandas as pd

In [2]:
# C2D with Quality flags for each column
c2d = Table.read('c2d.fit').to_pandas()

In [3]:
# Change the byte strings to regular strings and remove excess space
for colname in ['OType','q_FJ', 'q_FH', 'q_FKs', 'q_FIR1', 'q_FIR2', 'q_FIR3', 'q_FIR4', 'q_FMP1', 'q_FMP2']:
    c2d[colname] = c2d[colname].str.decode('utf-8')
    c2d[colname] = c2d[colname].str.rstrip()
# Rename object types in terms of classifications, and remove any ambiguous classifications
c2d.replace({'OType': r'^YSOc'}, {'OType': 0},regex=True,inplace=True)
c2d.replace({'OType': r'^star'}, {'OType': 2},regex=True,inplace=True)
c2d.replace({'OType': r'^Galc'}, {'OType': 1},regex=True,inplace=True)
c2d.rename(columns={"OType":"Target"},inplace=True)
c2d = c2d[pd.to_numeric(c2d['Target'], errors='coerce').notnull()]


print(f"YSO shape: {c2d[c2d['Target']==0].shape}")
print(f"EG shape: {c2d[c2d['Target']==1].shape}")
print(f"Star shape: {c2d[c2d['Target']==2].shape}")

# Remove unnecessary columns
c2d.drop(['Name', 'Cat', 'Region','recno'], axis=1,inplace=True)

c2d

YSO shape: (4038, 35)
EG shape: (6660, 35)
Star shape: (881015, 35)


,RAJ2000,DEJ2000,alpha,Target,FJ,e_FJ,q_FJ,FH,e_FH,q_FH,...,q_FIR3,FIR4,e_FIR4,q_FIR4,FMP1,e_FMP1,q_FMP1,FMP2,e_FMP2,q_FMP2
2,277.212616,0.859071,-2.32,2,0.410,0.054,B,1.0,0.073,B,...,B,0.086,0.052,U,-0.259,0.302,U,0.0,0.0,N
4,277.212616,0.665340,-2.28,2,0.699,0.056,A,2.0,0.084,A,...,A,0.307,0.044,B,0.011,0.242,U,0.0,0.0,U
9,277.126923,0.464996,-2.74,2,0.000,0.000,N,0.0,0.000,N,...,C,0.046,0.027,U,0.070,0.258,U,0.0,0.0,U
16,277.120972,0.430608,-2.16,2,0.461,0.000,U,1.0,0.077,B,...,A,0.188,0.044,C,-0.045,0.297,U,0.0,0.0,U
19,277.126404,0.792648,-2.48,2,2.290,0.078,A,4.0,0.140,A,...,A,0.642,0.054,A,-0.093,0.242,U,0.0,0.0,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6634891,67.516670,18.230413,-0.92,0,567.000,12.000,A,670.0,11.100,A,...,A,262.000,13.000,A,1210.000,112.000,A,0.0,0.0,U
6634892,75.777412,25.388849,-1.00,0,173.000,3.500,A,240.0,6.400,A,...,A,126.000,6.540,A,193.000,17.800,A,0.0,0.0,U
6634893,252.190079,-14.276626,-0.97,0,519.000,11.500,A,957.0,33.500,A,...,A,1010.000,56.400,A,1270.000,118.000,A,0.0,0.0,U
6634894,67.714104,24.706188,-1.21,0,254.000,4.910,A,341.0,9.100,A,...,A,115.000,5.820,A,118.000,10.900,A,0.0,0.0,U


In [4]:
# Find smallest reliable flux for each band and replace any flux tagged 'U' (for upper limit) with 1/100th of this to represent thermal noise.
columns = [idx for idx in c2d.columns.values if idx[0].lower() == 'F'.lower()]
for c in columns:
    min_f = min(c2d[c2d['q_'+c]=='A'][c])
    c2d.loc[c2d['q_'+c]=='N',c] = min_f/100
    c2d.loc[c2d['q_'+c]=='U',c] = min_f/100


In [5]:

def alpha_IR(f1,f2):
    y = 1.64*(f1-f2)-2.82
    return y


def mJy_to_mag(F, F0):
    m = -2.5*np.log10(F*0.001/F0)
    return m


def e_mJy_to_mag(e_F, F):
    m = (2.5/np.log(10))*(e_F/F)
    return m

# Change fluxes to Vega magnitudes
c2d.dropna()
c2d_fluxes = c2d[columns].values.astype(float).transpose() # Fluxes
e_columns = [idx for idx in c2d.columns.values if idx[0].lower() == 'e'.lower()]
c2d_fluxes_errs = c2d[e_columns].values.astype(float).transpose() #Errors
# Convert flux to magnitudes
for i, F_0 in enumerate([1594, 1024, 666.7, 277.500, 179.500, 116.600, 63.100, 7.140]):
    c2d[columns[i]] = mJy_to_mag(c2d_fluxes[i],F_0) # Convert fluxes in a column to mags
    c2d[e_columns[i]] = e_mJy_to_mag(c2d_fluxes_errs[i],c2d_fluxes[i]) # Convert errors in a column to mag


print(f"YSO shape: {c2d[c2d['Target']==0].shape}")
print(f"EG shape: {c2d[c2d['Target']==1].shape}")
print(f"Star shape: {c2d[c2d['Target']==2].shape}")
c2d.columns = ['RAJ2000', 'DEJ2000', 'alpha', 'Target', 'mag_J', 'e_mag_J', 'q_mag_J', 'mag_H',
       'e_mag_H', 'q_mag_H', 'mag_Ks', 'e_mag_Ks', 'q_mag_Ks', 'mag_IR1', 'e_mag_IR1', 'q_mag_IR1',
       'mag_IR2', 'e_mag_IR2', 'q_mag_IR2', 'mag_IR3', 'e_mag_IR3', 'q_mag_IR3', 'mag_IR4',
       'e_mag_IR4', 'q_mag_IR4', 'mag_MP1', 'e_mag_MP1', 'q_mag_MP1', 'mag_MP2', 'e_mag_MP2',
       'q_mag_MP2']
c2d.dropna(subset=['mag_IR1','e_mag_IR1','mag_IR2','e_mag_IR2','mag_IR3','e_mag_IR3','mag_IR4','e_mag_IR4'],inplace=True)

print(f"YSO shape: {c2d[c2d['Target']==0].shape}")
print(f"EG shape: {c2d[c2d['Target']==1].shape}")
print(f"Star shape: {c2d[c2d['Target']==2].shape}")
# Change column names to reflect these are magnitudes now


c2d

/var/folders/qy/7yz8bmv96hx8rndql_rfdz_80000gn/T/ipykernel_37760/3115989305.py:7: RuntimeWarning: divide by zero encountered in log10
  m = -2.5*np.log10(F*0.001/F0)
/var/folders/qy/7yz8bmv96hx8rndql_rfdz_80000gn/T/ipykernel_37760/3115989305.py:12: RuntimeWarning: divide by zero encountered in true_divide
  m = (2.5/np.log(10))*(e_F/F)
/var/folders/qy/7yz8bmv96hx8rndql_rfdz_80000gn/T/ipykernel_37760/3115989305.py:12: RuntimeWarning: invalid value encountered in true_divide
  m = (2.5/np.log(10))*(e_F/F)


YSO shape: (4038, 31)
EG shape: (6660, 31)
Star shape: (881015, 31)
YSO shape: (4038, 31)
EG shape: (6660, 31)
Star shape: (880985, 31)


,RAJ2000,DEJ2000,alpha,Target,mag_J,e_mag_J,q_mag_J,mag_H,e_mag_H,q_mag_H,...,q_mag_IR3,mag_IR4,e_mag_IR4,q_mag_IR4,mag_MP1,e_mag_MP1,q_mag_MP1,mag_MP2,e_mag_MP2,q_mag_MP2
2,277.212616,0.859071,-2.32,2,16.474261,0.142999,B,15.025750,0.079259,B,...,B,19.311206,47.443935,U,15.428883,68.168884,U,0.291,0.0,N
4,277.212616,0.665340,-2.28,2,15.895028,0.086983,A,14.273175,0.045601,A,...,A,13.282227,0.155610,B,15.428883,54.625397,U,0.291,0.0,U
9,277.126923,0.464996,-2.74,2,21.652273,0.000000,N,20.025750,0.000000,N,...,C,19.311206,24.634351,U,15.428883,58.236994,U,0.291,0.0,U
16,277.120972,0.430608,-2.16,2,21.652273,0.000000,U,15.025750,0.083602,B,...,A,13.814679,0.254108,C,15.428883,67.040260,U,0.291,0.0,U
19,277.126404,0.792648,-2.48,2,14.606632,0.036981,A,13.520600,0.038001,A,...,A,12.481236,0.091324,A,15.428883,54.625397,U,0.291,0.0,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6634891,67.516670,18.230413,-0.92,0,8.622263,0.022979,A,7.960563,0.017988,A,...,A,5.954320,0.053872,A,1.927282,0.100498,A,0.291,0.0,U
6634892,75.777412,25.388849,-1.00,0,9.911106,0.021966,A,9.075222,0.028953,A,...,A,6.749147,0.056355,A,3.920352,0.100135,A,0.291,0.0,U
6634893,252.190079,-14.276626,-0.97,0,8.718302,0.024058,A,7.573470,0.038006,A,...,A,4.489270,0.060629,A,1.874736,0.100879,A,0.291,0.0,U
6634894,67.714104,24.706188,-1.21,0,9.494137,0.020988,A,8.693864,0.028974,A,...,A,6.848329,0.054948,A,4.454541,0.100293,A,0.291,0.0,U


In [9]:
c2d.replace([np.inf, -np.inf], np.nan, inplace=True)
c2d.dropna(inplace=True)

print(f"YSO shape: {c2d[c2d['Target']==0].shape}")
print(f"EG shape: {c2d[c2d['Target']==1].shape}")
print(f"Star shape: {c2d[c2d['Target']==2].shape}")

YSO shape: (3959, 31)
EG shape: (6426, 31)
Star shape: (775866, 31)


In [10]:
c2d.to_csv('c2d_w_quality.csv')

In [8]:
np.all(np.isfinite(c2d[['mag_IR1','e_mag_IR1','mag_IR2','e_mag_IR2','mag_IR3','e_mag_IR3','mag_IR4','e_mag_IR4','mag_MP1','e_mag_MP1']].values))

True